In [2]:
import pandas as pd
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import numpy as np

In [31]:
#import csv
real_estate_file = 'data/nyc-rolling-sales.csv'
real_estate_df = pd.read_csv(real_estate_file)


real_estate_df.sample(50)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
3967,1,GREENWICH VILLAGE-CENTRAL,13 CONDOS - ELEVATOR APARTMENTS,2,569,1236,,R4,13 EAST 11TH STREET,3G,...,1,0,1,-,-,0,2,R4,-,10/28/16 0:00
73582,4,SPRINGFIELD GARDENS,01 ONE FAMILY DWELLINGS,1,12997,145,,A2,179-03 135TH AVENUE,,...,1,0,1,7633,1288,1945,1,A2,407000,7/11/17 0:00
14402,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,,1497,1213,,,60 EAST 86TH STREET,,...,0,0,0,-,-,0,2,R4,8604212,4/27/17 0:00
43179,3,OCEAN HILL,31 COMMERCIAL VACANT LAND,4,1574,21,,V1,14 HAVENS PLACE,,...,0,0,0,2300,0,0,4,V1,409000,4/6/17 0:00
67028,4,MASPETH,06 TAX CLASS 1 - OTHER,1,2389,1,,G0,48-51 65TH PLACE,,...,0,0,0,2000,-,1950,1,G0,-,2/13/17 0:00
12782,1,UPPER EAST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1400,1069,,R4,181 EAST 65TH STREET,21B,...,1,0,1,-,-,0,2,R4,6750000,6/9/17 0:00
14475,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1515,1067,,R4,110 EAST 87,PH-A,...,1,0,1,-,-,1939,2,R4,3999000,9/8/16 0:00
73077,4,SOUTH OZONE PARK,01 ONE FAMILY DWELLINGS,1,11781,279,,A1,131-15 135TH PLACE,,...,1,0,1,2100,840,1925,1,A1,266700,12/21/16 0:00
31416,3,BUSHWICK,02 TWO FAMILY DWELLINGS,1,3232,61,,B1,1281 DEKALB AVENUE,,...,2,0,2,1258,2448,2007,1,B1,830000,7/25/17 0:00
40283,3,GRAVESEND,03 THREE FAMILY DWELLINGS,1,7082,37,,C0,1974 WEST 6TH STREET,,...,3,0,3,2400,2136,1925,1,C0,1288000,12/2/16 0:00


In [4]:
# change address column type
real_estate_df = real_estate_df.astype({"ADDRESS": object})

real_estate_df.dtypes

BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
EASE-MENT                         object
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                        object
SALE DATE                         object
dtype: object

In [5]:
#change price column type

real_estate_df['SALE PRICE'] = real_estate_df['SALE PRICE'].fillna(0)

# real_estate_df['SALE PRICE'].str.replace('-','0')

real_estate_df['SALE PRICE'] = real_estate_df['SALE PRICE'].str.replace('-', "0")

real_estate_df['SALE PRICE'] = real_estate_df['SALE PRICE'].astype(str).astype(int)

real_estate_df.sample(50)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
54734,4,ELMHURST,02 TWO FAMILY DWELLINGS,1,1524,59,,B1,42-71 78TH STREET,,...,2,0,2,3500,2420,1920,1,B1,1538000,6/23/17 0:00
17082,1,UPPER WEST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1252,75,,D4,"214 RIVERSIDE DRIVE, 214",,...,0,0,0,-,-,1901,2,D4,1260000,5/24/17 0:00
1815,1,EAST VILLAGE,08 RENTALS - ELEVATOR APARTMENTS,2,436,44,,D3,107 ST MARKS PLACE,,...,48,0,48,8904,25610,1920,2,D3,30449204,8/8/17 0:00
83396,5,TOTTENVILLE,02 TWO FAMILY DWELLINGS,1,7895,42,,B2,311 YETMAN AVENUE,,...,2,0,2,3856,2160,2015,1,B2,0,12/21/16 0:00
61243,4,FRESH MEADOWS,09 COOPS - WALKUP APARTMENTS,2,7117,51,,C6,"196-11 69TH AVENUE, 2",,...,0,0,0,-,-,1952,2,C6,175000,7/24/17 0:00
37825,3,FLATBUSH-EAST,01 ONE FAMILY DWELLINGS,1,4960,35,,A5,1238 TROY AVENUE,,...,1,0,1,1900,1216,1930,1,A5,0,5/4/17 0:00
57657,4,FLUSHING-NORTH,10 COOPS - ELEVATOR APARTMENTS,2,5137,11,,D4,"42-55 COLDEN STREET, 10H",,...,0,0,0,-,-,1962,2,D4,0,4/3/17 0:00
74724,4,SUNNYSIDE,10 COOPS - ELEVATOR APARTMENTS,2,177,1,,D4,"48-35 43RD STREET, 6E",,...,0,0,0,-,-,1931,2,D4,356000,7/25/17 0:00
16581,1,UPPER WEST SIDE (79-96),09 COOPS - WALKUP APARTMENTS,2C,1211,23,,C6,"129 WEST 80TH STREET, 4",,...,0,0,0,-,-,1900,2,C6,1625000,2/17/17 0:00
1190,1,CIVIC CENTER,13 CONDOS - ELEVATOR APARTMENTS,2,123,1172,,R4,30 PARK PLACE,52D,...,1,0,1,-,-,2008,2,R4,4459935,4/10/17 0:00


In [6]:
real_estate_df.dtypes

BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
EASE-MENT                         object
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                         int64
SALE DATE                         object
dtype: object

In [7]:
#add borough depending on number definfed by the data documentation.
borough_name = []

for b in real_estate_df['BOROUGH']:
    if  b == 1:
        borough_name.append("Manhattan")
        
    elif b == 2:
        borough_name.append("Bronx") 
        
    elif b == 3: 
        borough_name.append("Brooklyn") 
        
    elif b == 4:
        borough_name.append("Queens")
        
    else:
        borough_name.append("Staten Island") 
        

real_estate_df["borough_name"] = borough_name
real_estate_df.sample(50)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,borough_name
56760,4,FLUSHING-NORTH,01 ONE FAMILY DWELLINGS,1,7066,39,,A5,58-08 183RD STREET,,...,0,1,2400,1520,1960,1,A5,0,12/16/16 0:00,Queens
15813,1,UPPER WEST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1150,1276,,R4,172 WEST 79 STREET,14C,...,0,1,-,-,0,2,R4,0,2/8/17 0:00,Manhattan
52695,4,BRIARWOOD,13 CONDOS - ELEVATOR APARTMENTS,2,9712,1130,,R4,141-16 84TH DRIVE,5B,...,0,1,-,-,2009,2,R4,490000,4/28/17 0:00,Queens
1074,1,CHINATOWN,15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,172,1306,,R1,77 WHITE STREET,6,...,0,1,-,-,0,2,R1,0,3/24/17 0:00,Manhattan
73205,4,SOUTH OZONE PARK,02 TWO FAMILY DWELLINGS,1,11672,6,,B3,128-14 115TH AVENUE,,...,0,2,4000,1952,1940,1,B3,580000,11/21/16 0:00,Queens
29248,3,BOROUGH PARK,02 TWO FAMILY DWELLINGS,1,5621,13,,S2,4624 NEW UTRECHT AVENUE,,...,1,3,1537,3000,1931,1,S2,0,5/17/17 0:00,Brooklyn
78029,5,GRANT CITY,01 ONE FAMILY DWELLINGS,1,3535,95,,A5,23 STOBE AVENUE,,...,0,1,2761,1870,1970,1,A5,470000,2/24/17 0:00,Staten Island
51928,4,BAYSIDE,13 CONDOS - ELEVATOR APARTMENTS,2,5893,1834,,R4,2 BAY CLUB DRIVE,14W,...,0,1,-,-,1972,2,R4,890000,1/20/17 0:00,Queens
84132,5,WESTERLEIGH,02 TWO FAMILY DWELLINGS,1,1501,272,,B2,49 CAMBRIDGE AVENUE,,...,0,2,4680,2340,1970,1,B2,812500,8/2/17 0:00,Staten Island
69745,4,REGO PARK,10 COOPS - ELEVATOR APARTMENTS,2,3173,70,,D4,"67-87 BOOTH STREET, 6B",,...,0,0,-,-,1950,2,D4,550000,7/13/17 0:00,Queens


In [8]:
#get rid of appt numbers
real_estate_df['ADDRESS'] = real_estate_df['ADDRESS'].str.split(",", n = 1, expand = True)


real_estate_df.sample(50)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,borough_name
77151,5,CASTLETON CORNERS,01 ONE FAMILY DWELLINGS,1,377,15,,A2,215 KEMBALL AVENUE,,...,0,1,3500,1300,1964,1,A2,500000,2/28/17 0:00,Staten Island
84412,5,WILLOWBROOK,44 CONDO PARKING,4,1965,1154,,RG,1235 FOREST HILL ROAD,P31,...,0,1,-,-,2015,4,RG,0,6/1/17 0:00,Staten Island
32522,3,CANARSIE,02 TWO FAMILY DWELLINGS,1,8173,12,,B2,661 EAST 104 STREET,,...,0,2,1600,1760,2005,1,B2,594000,3/9/17 0:00,Brooklyn
9342,1,MIDTOWN WEST,13 CONDOS - ELEVATOR APARTMENTS,,1042,1616,,,318-322 WEST 52ND STREET,,...,0,0,-,-,0,2,R4,1756481,7/21/17 0:00,Manhattan
16831,1,UPPER WEST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1224,1,,D4,175 WEST 93RD STREET,,...,0,0,-,-,1929,2,D4,1800000,6/30/17 0:00,Manhattan
47534,3,WILLIAMSBURG-CENTRAL,02 TWO FAMILY DWELLINGS,1,2226,12,,B3,178 RUTLEDGE STREET,,...,0,2,1900,2460,1899,1,B3,0,1/1/17 0:00,Brooklyn
51107,4,BAYSIDE,01 ONE FAMILY DWELLINGS,1,7334,12,,A5,45-14 215 PLACE,,...,0,1,2017,1448,1930,1,A5,760000,7/20/17 0:00,Queens
20783,2,MELROSE/CONCOURSE,02 TWO FAMILY DWELLINGS,1,2418,20,,B3,813 COURTLANDT AVENUE,,...,0,2,2450,2000,1901,1,B3,439000,11/10/16 0:00,Bronx
82396,5,ROSSVILLE,01 ONE FAMILY DWELLINGS,1,7086,58,,A1,137 LUCILLE AVENUE,,...,0,1,3600,1776,1985,1,A1,570000,4/27/17 0:00,Staten Island
44040,3,OCEAN PARKWAY-SOUTH,10 COOPS - ELEVATOR APARTMENTS,2,7159,40,,D4,735 AVENUE W,,...,0,0,0,0,1962,2,D4,245000,10/20/16 0:00,Brooklyn


In [9]:
#making new column with full address

real_estate_df["Full_Address"] = real_estate_df["ADDRESS"] + " "  + real_estate_df["borough_name"]

real_estate_df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,borough_name,Full_Address
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,...,5,1633,6440,1900,2,C2,6625000,7/19/17 0:00,Manhattan,153 AVENUE B Manhattan
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,...,31,4616,18690,1900,2,C7,0,12/14/16 0:00,Manhattan,234 EAST 4TH STREET Manhattan
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,...,17,2212,7803,1900,2,C7,0,12/9/16 0:00,Manhattan,197 EAST 3RD STREET Manhattan
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,...,10,2272,6794,1913,2,C4,3936272,9/23/16 0:00,Manhattan,154 EAST 7TH STREET Manhattan
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,...,6,2369,4615,1900,2,C2,8000000,11/17/16 0:00,Manhattan,301 EAST 10TH STREET Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,...,2,2400,2575,1998,1,B9,450000,11/28/16 0:00,Staten Island,37 QUAIL LANE Staten Island
84544,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,...,2,2498,2377,1998,1,B9,550000,4/21/17 0:00,Staten Island,32 PHEASANT LANE Staten Island
84545,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,...,2,4000,1496,1925,1,B2,460000,7/5/17 0:00,Staten Island,49 PITNEY AVENUE Staten Island
84546,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,...,7,208033,64117,2001,4,K6,11693337,12/21/16 0:00,Staten Island,2730 ARTHUR KILL ROAD Staten Island


In [14]:
# real_estate_dedupe = df.groupby(["Full_Address"])


# real_estate_df.groupby(['Full_Address', 'SALE PRICE']).mean()

real_estate_clean_df = real_estate_df.groupby(['Full_Address'], as_index=False).agg({'SALE PRICE': 'mean', 
                                                                                     'NEIGHBORHOOD': 'first', 
                                                                                     'TAX CLASS AT PRESENT': 'first', 
                                                                                     'ZIP CODE': 'first', 
                                                                                     'BUILDING CLASS AT TIME OF SALE': 'first', 
                                                                                     'borough_name': 'first'})

real_estate_clean_df.head(15)

,Full_Address,SALE PRICE,NEIGHBORHOOD,TAX CLASS AT PRESENT,ZIP CODE,BUILDING CLASS AT TIME OF SALE,borough_name
0,****** 95TH STREET Queens,0.000000e+00,OZONE PARK,1B,11417,V0,Queens
1,1 12TH ST EXTENSION Brooklyn,0.000000e+00,GOWANUS,4,11215,K6,Brooklyn
2,1 5 AVENUE Manhattan,4.925000e+06,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan
3,1 5TH AVENUE Manhattan,0.000000e+00,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan
4,1 ASCAN AVE Queens,6.490000e+05,FOREST HILLS,2,11375,D4,Queens
5,1 ASCAN AVENUE Queens,3.950000e+05,FOREST HILLS,2,11375,D4,Queens
6,1 BAY CLUB DRIVE Queens,5.264299e+05,BAYSIDE,2,11360,R4,Queens
7,1 BAYCLUB DRIVE Queens,0.000000e+00,BAYSIDE,2,11360,R4,Queens
8,1 BEEKMAN PLACE Manhattan,0.000000e+00,MIDTOWN EAST,2,10022,D4,Manhattan
9,1 BOERUM PLACE Brooklyn,7.650000e+07,DOWNTOWN-FULTON MALL,4,11201,O2,Brooklyn


In [15]:
#picking columns and assigning to new dataframe
# groupby_df = ['Full_Address','NEIGHBORHOOD', 'TAX CLASS AT PRESENT',
#                    'ZIP CODE', 'SALE PRICE', 'BUILDING CLASS AT TIME OF SALE', 'borough_name']

# real_estate_clean_df = real_estate_df[real_estate_cols]

# real_estate_clean_df

In [16]:
real_estate_clean_df.apply(lambda col: col.drop_duplicates().reset_index(drop=True))


real_estate_clean_df

,Full_Address,SALE PRICE,NEIGHBORHOOD,TAX CLASS AT PRESENT,ZIP CODE,BUILDING CLASS AT TIME OF SALE,borough_name
0,****** 95TH STREET Queens,0.0,OZONE PARK,1B,11417,V0,Queens
1,1 12TH ST EXTENSION Brooklyn,0.0,GOWANUS,4,11215,K6,Brooklyn
2,1 5 AVENUE Manhattan,4925000.0,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan
3,1 5TH AVENUE Manhattan,0.0,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan
4,1 ASCAN AVE Queens,649000.0,FOREST HILLS,2,11375,D4,Queens
...,...,...,...,...,...,...,...
58586,WOODROW ROAD Staten Island,0.0,ARDEN HEIGHTS,1B,10312,V0,Staten Island
58587,WOODYCREST AVENUE Bronx,750000.0,HIGHBRIDGE/MORRIS HEIGHTS,4,0,G7,Bronx
58588,WORTMAN AVENUE Brooklyn,0.0,EAST NEW YORK,4,0,G7,Brooklyn
58589,YORK AVENUE Staten Island,235000.0,NEW BRIGHTON,1B,10301,V0,Staten Island


In [17]:
#create lat and long columns 

geolocator = Nominatim(user_agent="user_agent")
location = geolocator.geocode("327 EAST 3rd STREET Manhattan")

print((location.latitude, location.longitude))

(40.7208628, -73.9784408)


In [20]:


lat = []
long = []

i = 0

geolocator = Nominatim(user_agent="user_agent")

for address in real_estate_clean_df["Full_Address"]:
    
    addr = geolocator.geocode(address, timeout= 2)  
    if addr == None:
        lat.append(np.nan)
        long.append(np.nan)
        i += 1
        continue 
        
    else:
        i += 1
        lat.append(addr.latitude)
        long.append(addr.longitude)
        
    
    if i%500 == 0:
        print(addr.latitude, addr.longitude)
        
    else:
        pass
    


real_estate_clean_df['latitude'] = lat
real_estate_clean_df['longitude'] = long

40.6865818 -73.9683503
40.626845450000005 -74.01023305
40.8273763 -73.86118369275668
40.6544339 -73.8367169
40.6968643 -73.7853514
40.7871559 -73.83804072147147
40.754153200000005 -73.8559273689875
40.63236705 -73.92613370000001
40.676400349999994 -73.82227416392092
40.6951942 -73.8442743
40.74389664177796 -73.99287817469734
40.7461719 -73.9883736
40.781851700000004 -73.97689972304559
-36.7839272 175.0291945
40.5958892 -74.09529493966245
40.66936565 -73.80246330018936
40.761103399999996 -73.83316310990327
40.630420349999994 -73.99462299947507
40.7310922 -73.8281964
40.588130500000005 -74.18851971981422
40.789201649999995 -73.77235775076846
40.7546729 -73.8331797
40.694710900000004 -73.80588792478194
40.7048638 -73.8009766
40.7590997 -73.96945786056415
40.692827550000004 -73.90268329669064
40.6484581 -73.94958410171886
40.7741514 -73.8061286
40.730996 -73.879686
40.633865400000005 -73.8914171621758
40.7223469 -74.00851108026316
40.6756771 -73.93112055
40.587113200000005 -73.967031577504

In [ ]:
real_estate_clean_df[real_estate_clean_df['Full_Address'].str.contains('1 5TH AVENUE')]

In [26]:
real_estate_clean_df


,Full_Address,SALE PRICE,NEIGHBORHOOD,TAX CLASS AT PRESENT,ZIP CODE,BUILDING CLASS AT TIME OF SALE,borough_name,latitude,longitude
0,****** 95TH STREET Queens,0.0,OZONE PARK,1B,11417,V0,Queens,40.686989,-73.847231
1,1 12TH ST EXTENSION Brooklyn,0.0,GOWANUS,4,11215,K6,Brooklyn,NaN,NaN
2,1 5 AVENUE Manhattan,4925000.0,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan,49.836689,24.020745
3,1 5TH AVENUE Manhattan,0.0,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan,40.745898,-73.986688
4,1 ASCAN AVE Queens,649000.0,FOREST HILLS,2,11375,D4,Queens,40.717455,-73.841313
...,...,...,...,...,...,...,...,...,...
58586,WOODROW ROAD Staten Island,0.0,ARDEN HEIGHTS,1B,10312,V0,Staten Island,40.546672,-74.191896
58587,WOODYCREST AVENUE Bronx,750000.0,HIGHBRIDGE/MORRIS HEIGHTS,4,0,G7,Bronx,40.830083,-73.929969
58588,WORTMAN AVENUE Brooklyn,0.0,EAST NEW YORK,4,0,G7,Brooklyn,40.661459,-73.874470
58589,YORK AVENUE Staten Island,235000.0,NEW BRIGHTON,1B,10301,V0,Staten Island,40.642265,-74.087684


In [22]:
#push to new csv 

real_estate_clean_df.to_csv('nyc_real_estate_lat_long.csv', index = False)

In [28]:
#drop nans
final_real_estate_df = real_estate_clean_df.dropna()


final_real_estate_df 


,Full_Address,SALE PRICE,NEIGHBORHOOD,TAX CLASS AT PRESENT,ZIP CODE,BUILDING CLASS AT TIME OF SALE,borough_name,latitude,longitude
0,****** 95TH STREET Queens,0.0,OZONE PARK,1B,11417,V0,Queens,40.686989,-73.847231
2,1 5 AVENUE Manhattan,4925000.0,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan,49.836689,24.020745
3,1 5TH AVENUE Manhattan,0.0,GREENWICH VILLAGE-CENTRAL,2,10003,D4,Manhattan,40.745898,-73.986688
4,1 ASCAN AVE Queens,649000.0,FOREST HILLS,2,11375,D4,Queens,40.717455,-73.841313
5,1 ASCAN AVENUE Queens,395000.0,FOREST HILLS,2,11375,D4,Queens,40.717455,-73.841313
...,...,...,...,...,...,...,...,...,...
58586,WOODROW ROAD Staten Island,0.0,ARDEN HEIGHTS,1B,10312,V0,Staten Island,40.546672,-74.191896
58587,WOODYCREST AVENUE Bronx,750000.0,HIGHBRIDGE/MORRIS HEIGHTS,4,0,G7,Bronx,40.830083,-73.929969
58588,WORTMAN AVENUE Brooklyn,0.0,EAST NEW YORK,4,0,G7,Brooklyn,40.661459,-73.874470
58589,YORK AVENUE Staten Island,235000.0,NEW BRIGHTON,1B,10301,V0,Staten Island,40.642265,-74.087684


In [29]:

final_real_estate_df.to_csv('data/nyc_real_estate_cleaned_final.csv', index = False)

In [30]:
final_real_estate_df.dtypes

Full_Address                       object
SALE PRICE                        float64
NEIGHBORHOOD                       object
TAX CLASS AT PRESENT               object
ZIP CODE                            int64
BUILDING CLASS AT TIME OF SALE     object
borough_name                       object
latitude                          float64
longitude                         float64
dtype: object

In [37]:
real_estate_df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [42]:




data= [final_real_estate_df["Full_Address"], real_estate_df["BUILDING CLASS CATEGORY"], 
       final_real_estate_df["SALE PRICE"],final_real_estate_df["NEIGHBORHOOD"],
       final_real_estate_df["TAX CLASS AT PRESENT"],final_real_estate_df["ZIP CODE"],
       final_real_estate_df["BUILDING CLASS AT TIME OF SALE"],
       final_real_estate_df["borough_name"],final_real_estate_df["latitude"],
       final_real_estate_df["longitude"]]

building_type_df = pd.concat(data, axis=1)

building_type_df

,Full_Address,BUILDING CLASS CATEGORY,SALE PRICE,NEIGHBORHOOD,TAX CLASS AT PRESENT,ZIP CODE,BUILDING CLASS AT TIME OF SALE,borough_name,latitude,longitude
0,****** 95TH STREET Queens,07 RENTALS - WALKUP APARTMENTS,0.0,OZONE PARK,1B,11417.0,V0,Queens,40.686989,-73.847231
1,NaN,07 RENTALS - WALKUP APARTMENTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 5 AVENUE Manhattan,07 RENTALS - WALKUP APARTMENTS,4925000.0,GREENWICH VILLAGE-CENTRAL,2,10003.0,D4,Manhattan,49.836689,24.020745
3,1 5TH AVENUE Manhattan,07 RENTALS - WALKUP APARTMENTS,0.0,GREENWICH VILLAGE-CENTRAL,2,10003.0,D4,Manhattan,40.745898,-73.986688
4,1 ASCAN AVE Queens,07 RENTALS - WALKUP APARTMENTS,649000.0,FOREST HILLS,2,11375.0,D4,Queens,40.717455,-73.841313
...,...,...,...,...,...,...,...,...,...,...
84543,NaN,02 TWO FAMILY DWELLINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84544,NaN,02 TWO FAMILY DWELLINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84545,NaN,02 TWO FAMILY DWELLINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84546,NaN,22 STORE BUILDINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
final_w_building_type = building_type_df.dropna().reset_index()


final_w_building_type = final_w_building_type.rename(columns={"Full_Address": "full_address",
                                               "BUILDING CLASS CATEGORY": "building_class_category",
                                               "SALE PRICE": "sale_price",
                                               "NEIGHBORHOOD": "neighborhood",
                                               "TAX CLASS AT PRESENT": "tax_class_at_present",
                                               "ZIP CODE": "zip_code",
                                               "BUILDING CLASS AT TIME OF SALE": "building_class_at_time_of_sale",
                                               "borough_name": "borough_name",
                                               "latitude": "latitude",
                                               "longitude": "longitude"})

final_w_building_type

,index,full_address,building_class_category,sale_price,neighborhood,tax_class_at_present,zip_code,building_class_at_time_of_sale,borough_name,latitude,longitude
0,0,****** 95TH STREET Queens,07 RENTALS - WALKUP APARTMENTS,0.0,OZONE PARK,1B,11417.0,V0,Queens,40.686989,-73.847231
1,2,1 5 AVENUE Manhattan,07 RENTALS - WALKUP APARTMENTS,4925000.0,GREENWICH VILLAGE-CENTRAL,2,10003.0,D4,Manhattan,49.836689,24.020745
2,3,1 5TH AVENUE Manhattan,07 RENTALS - WALKUP APARTMENTS,0.0,GREENWICH VILLAGE-CENTRAL,2,10003.0,D4,Manhattan,40.745898,-73.986688
3,4,1 ASCAN AVE Queens,07 RENTALS - WALKUP APARTMENTS,649000.0,FOREST HILLS,2,11375.0,D4,Queens,40.717455,-73.841313
4,5,1 ASCAN AVENUE Queens,07 RENTALS - WALKUP APARTMENTS,395000.0,FOREST HILLS,2,11375.0,D4,Queens,40.717455,-73.841313
...,...,...,...,...,...,...,...,...,...,...,...
55737,58586,WOODROW ROAD Staten Island,13 CONDOS - ELEVATOR APARTMENTS,0.0,ARDEN HEIGHTS,1B,10312.0,V0,Staten Island,40.546672,-74.191896
55738,58587,WOODYCREST AVENUE Bronx,13 CONDOS - ELEVATOR APARTMENTS,750000.0,HIGHBRIDGE/MORRIS HEIGHTS,4,0.0,G7,Bronx,40.830083,-73.929969
55739,58588,WORTMAN AVENUE Brooklyn,13 CONDOS - ELEVATOR APARTMENTS,0.0,EAST NEW YORK,4,0.0,G7,Brooklyn,40.661459,-73.874470
55740,58589,YORK AVENUE Staten Island,13 CONDOS - ELEVATOR APARTMENTS,235000.0,NEW BRIGHTON,1B,10301.0,V0,Staten Island,40.642265,-74.087684


In [51]:
final_w_building_type.to_csv('data/nyc_real_estate_building_final_use_this.csv', index = False)